# 구인구직 사이트에서 "개발자" 채용 목록 가져오기(Wanted)
* Target URL: https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=job.latest_order&years=-1&locations=all
* 가져올 정보: 채용 공고 제목, 회사명, 응답률, 위치, 채용 보상금
* 데이터 수집 방법: Selenium을 이용한 크롤링 -> BeautifulSoup을 이용한 파싱 -> Pandas DataFrame을 이용한 정형화

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome('./chromedriver')

driver.get("https://www.wanted.co.kr/wdlist/518")
print(driver.title[:10])

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "#__next > div > div._1yHloYOs_bDD0E-s121Oaa > div._2y4sIVmvSrf6Iy63okz9Qh > div > ul > li:nth-child(1) > div > a > div"))
)
last_height = driver.execute_script("return document.body.scrollHeight")
for i in range(10): # 끝까지 스크롤 10번을 하여 총 100개의 채용 공고 크롤링

    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

    # 채용 정보 로딩 완료까지 최대 10초 기다리기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#__next > div > div._1yHloYOs_bDD0E-s121Oaa > div._2y4sIVmvSrf6Iy63okz9Qh > div > ul > li:nth-child(1) > div > a > div"))
    )

html = driver.page_source
driver.close()

개발 채용 정보 |


In [8]:
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
soup = BeautifulSoup(html)
jobs = soup.select('._3D4OeuZHyGXN7wwibRM5BJ') # job-card 클래스명으로 select
jobs[1].select('.body')

[<div class="body"><div class="job-card-position">[CLO3D] Software QA Engineer</div><div class="job-card-company-name">클로버추얼패션(CLOVirtualFashion)</div><div class="job-card-company-location">서울<span class="addressDot">.</span><span>한국</span></div><div class="reward">채용보상금 1,000,000원</div></div>]

In [10]:
job_list = []
for job in jobs:
    try:
        position = job.select('.job-card-position')[0].get_text()
        company = job.select('.job-card-company-name')[0].get_text()
        location = job.select('.job-card-company-location')[0].get_text()
        reward = job.select('.reward')[0].get_text()[6:-1].replace(',','') # Reward(보상금)을 계산 가능한 수치(int or float)로 변환하기 위한 정형화

        job_list.append([position, company, location, int(reward)])
    except:
        print(job)
        break

job_list

[['서버 개발자', '바디스앤소울스', '서울.한국', 1000000],
 ['[CLO3D] Software QA Engineer',
  '클로버추얼패션(CLOVirtualFashion)',
  '서울.한국',
  1000000],
 ['[AI홈트] Front-End 개발', '아이픽셀(iPIXEL)', '서울.한국', 1000000],
 ['[AI홈트] 백엔드 개발자', '아이픽셀(iPIXEL)', '서울.한국', 1000000],
 ['[이커머스] 클라우드 엔지니어(시스템)', '이랜드이노플', '서울.한국', 1000000],
 ['[이커머스] DW & BI 엔지니어', '이랜드이노플', '서울.한국', 1000000],
 ['Backend Engineer (Backend Lead)', '엔코위더스', '경기.한국', 1000000],
 ['Backend Engineer (Junior)', '엔코위더스', '경기.한국', 1000000],
 ['Frontend Engineer (Junior)', '엔코위더스', '경기.한국', 1000000],
 ['[이커머스] DBA', '이랜드이노플', '서울.한국', 1000000],
 ['[이커머스] JAVA서버 개발', '이랜드이노플', '서울.한국', 1000000],
 ['[이커머스] UI 개발자', '이랜드이노플', '서울.한국', 1000000],
 ['[EPAY] 이랜드그룹 페이 시스템 개발자', '이랜드이노플', '서울.한국', 1000000],
 ['[이커머스] 빅데이터 플랫폼 구축', '이랜드이노플', '서울.한국', 1000000],
 ['[SAP/ERP] FCM(FI/CO Module)', '이랜드이노플', '서울.한국', 1000000],
 ['프론트엔드 엔지니어(Frontend Engineer)', '라포랩스(퀸잇)', '서울.한국', 1000000],
 ['백엔드 엔지니어(Backend Engineer)', '라포랩스(퀸잇)', '서울.한국', 1000000],
 ['백엔드 개발자', '

In [11]:
columns = ['position','company','location','reward']
df = pd.DataFrame(job_list, columns=columns)
df

,position,company,location,reward
0,서버 개발자,바디스앤소울스,서울.한국,1000000
1,[CLO3D] Software QA Engineer,클로버추얼패션(CLOVirtualFashion),서울.한국,1000000
2,[AI홈트] Front-End 개발,아이픽셀(iPIXEL),서울.한국,1000000
3,[AI홈트] 백엔드 개발자,아이픽셀(iPIXEL),서울.한국,1000000
4,[이커머스] 클라우드 엔지니어(시스템),이랜드이노플,서울.한국,1000000
...,...,...,...,...
135,Software Engineer (Back-end) - (산업기능요원),크리에이트립,서울.한국,1000000
136,Software Engineer (Front-end) - (산업기능요원),크리에이트립,서울.한국,1000000
137,Software Engineer (Front-end),크리에이트립,서울.한국,1000000
138,Software Engineer (Back-end),크리에이트립,서울.한국,1000000
